In [176]:
import pandas as pd
import pickle
import math
import numpy as np
from sklearn.preprocessing import LabelEncoder

In [177]:
pd.options.display.float_format = '{:,.2f}'.format

In [178]:
kaggle_df = pd.read_csv("../proc_data/CombinedResults.csv")

espn_file = open("../pickled_data/espn_df.pickle", "rb")
espn_df = pickle.load(espn_file)
espn_file.close()

In [179]:
with open("../pickled_data/team_dict.pickle", "rb") as dict_file:
    team_dict = pickle.load(dict_file)
dict_file.close()

In [180]:
# taken from other project, will cite at end

mean_elo = 1600
elo_width = 400
k_factor = 64


def update_elo(winner_elo, loser_elo):
    expected_win = expected_result(winner_elo, loser_elo)
    change_in_elo = k_factor * (1 - expected_win)
    winner_elo += change_in_elo
    loser_elo -= change_in_elo
    return winner_elo, loser_elo


def expected_result(elo_a, elo_b):
    """
    https://en.wikipedia.org/wiki/Elo_rating_system#Mathematical_details
    """
    expect_a = 1.0/(1+10**((elo_b - elo_a)/elo_width))
    return expect_a


def update_end_of_season(elos):
    """Regression towards the mean
    
    Following 538 nfl methods
    https://fivethirtyeight.com/datalab/nfl-elo-ratings-are-back/
    """
    diff_from_mean = elos - mean_elo
    elos -= diff_from_mean/3
    return elos

In [181]:
# Sourced from Kasper P Lauritzen on ML madness discussion boards

kaggle_df['w_elo_before_game'] = 0
kaggle_df['w_elo_after_game'] = 0
kaggle_df['l_elo_before_game'] = 0
kaggle_df['l_elo_after_game'] = 0

elo_per_season = {}
n_teams = len(team_dict.keys())
current_elos = np.ones(shape=(n_teams)) * mean_elo
print n_teams

364


In [182]:
le = LabelEncoder()
kaggle_df["W_index"] = le.fit_transform(kaggle_df.Wteam)
kaggle_df["L_index"] = le.fit_transform(kaggle_df.Lteam)

In [183]:
kaggle_df.head(10)

,Unnamed: 0,Season,Daynum,Wteam,Wscore,Lteam,Lscore,Wor,Wdr,Wast,...,Lfgm3%,Lftm%,W_+/-,L_+/-,w_elo_before_game,w_elo_after_game,l_elo_before_game,l_elo_after_game,W_index,L_index
0,23662,2008,0,1272,102,1404,71,21,37,23,...,27.03,59.09,1,14,0,0,0,0,165,294
1,23663,2008,0,1350,44,1263,42,7,14,9,...,9.09,60.00,5,20,0,0,0,0,240,156
2,23664,2008,1,1205,69,1105,55,20,22,15,...,22.22,65.38,9,23,0,0,0,0,99,4
3,23665,2008,1,1246,67,1146,40,18,26,11,...,13.64,80.95,6,13,0,0,0,0,139,40
4,23666,2008,1,1272,80,1350,63,12,25,15,...,36.36,58.33,5,10,0,0,0,0,165,240
5,23667,2008,1,1404,66,1263,62,6,24,14,...,35.00,60.00,4,14,0,0,0,0,294,156
6,23668,2008,2,1146,69,1105,59,17,37,8,...,43.75,52.63,19,9,0,0,0,0,40,4
7,23669,2008,2,1163,69,1288,65,14,29,14,...,50.00,44.44,9,3,0,0,0,0,57,181
8,23670,2008,2,1205,84,1246,68,6,31,16,...,31.58,90.00,14,5,0,0,0,0,99,139
9,23671,2008,3,1163,82,1138,57,15,39,14,...,14.29,55.56,11,10,0,0,0,0,57,32


In [184]:
kaggle_df['total_days'] = (kaggle_df.Season-1970)*365.25 + kaggle_df.Daynum

In [185]:
df_team_elos = pd.DataFrame(index=kaggle_df.total_days.unique(), 
                            columns=range(n_teams))
df_team_elos.iloc[0, :] = current_elos

In [186]:
current_season = kaggle_df.at[0, "Season"]

for row in kaggle_df.itertuples():
    if row.Season != current_season:
        # Check if we are starting a new season. 
        # Regress all ratings towards the mean - lot of turnover every year
        current_elos = update_end_of_season(current_elos)
        # Write the beginning of new season ratings to a dict for later lookups.
        elo_per_season[row.Season] = current_elos.copy()
        current_season = row.Season
    idx = row.Index
    w_id = row.W_index
    l_id = row.L_index
    # Get current elos
    w_elo_before = current_elos[w_id]
    l_elo_before = current_elos[l_id]
    # Update on game results
    w_elo_after, l_elo_after = update_elo(w_elo_before, l_elo_before)
        
    # Save updated elos
    kaggle_df.at[idx, 'w_elo_before_game'] = w_elo_before
    kaggle_df.at[idx, 'l_elo_before_game'] = l_elo_before
    kaggle_df.at[idx, 'w_elo_after_game'] = w_elo_after
    kaggle_df.at[idx, 'l_elo_after_game'] = l_elo_after
    current_elos[w_id] = w_elo_after
    current_elos[l_id] = l_elo_after
    
    # Save elos to team DataFrame
    today = row.total_days
    df_team_elos.at[today, w_id] = w_elo_after
    df_team_elos.at[today, l_id] = l_elo_after

In [187]:
kaggle_df.drop(["W_index", "L_index", "total_days"], axis=1, inplace=True)
kaggle_df.drop(kaggle_df.columns[0], axis=1, inplace=True)
kaggle_df.head()

,Season,Daynum,Wteam,Wscore,Lteam,Lscore,Wor,Wdr,Wast,Wto,...,Wftm%,Lfgm%,Lfgm3%,Lftm%,W_+/-,L_+/-,w_elo_before_game,w_elo_after_game,l_elo_before_game,l_elo_after_game
0,2008,0,1272,102,1404,71,21,37,23,10,...,58.62,31.17,27.03,59.09,1,14,1600,1632,1600,1568
1,2008,0,1350,44,1263,42,7,14,9,18,...,85.71,42.22,9.09,60.00,5,20,1600,1632,1600,1568
2,2008,1,1205,69,1105,55,20,22,15,21,...,68.75,34.69,22.22,65.38,9,23,1600,1632,1600,1568
3,2008,1,1246,67,1146,40,18,26,11,16,...,68.75,20.00,13.64,80.95,6,13,1600,1632,1600,1568
4,2008,1,1272,80,1350,63,12,25,15,18,...,65.62,38.10,36.36,58.33,5,10,1632,1664,1632,1600


In [188]:
espn_df.dropna(how="any", inplace=True)
espn_df.head()

,Season,Teamname,SOR RK,SOS RK,Non-Conf SOS RK,BPI,Q_wins,Q_losses
0,2008,Air Force,162,155,307,-1.10,1,4
1,2008,Akron,77,138,133,6.60,0,1
2,2008,Alabama,97,63,135,6.00,4,10
3,2008,Alabama A&M,312,336,205,-10.10,0,1
4,2008,Alabama St,205,334,210,-4.30,0,1


In [189]:
print espn_df[espn_df["Teamname"] == "Air Force"]

      Season   Teamname  SOR RK  SOS RK  Non-Conf SOS RK   BPI Q_wins Q_losses
0       2008  Air Force     162     155              307 -1.10      1        4
341     2009  Air Force     287     166              337 -7.10      0        3
688     2010  Air Force     249     118              298 -7.10      0        8
1035    2011  Air Force     138      90              263 -1.70      0        7
1380    2012  Air Force     182     120              315 -1.30      0        6
1723    2013  Air Force      91      80              145  0.70      4        9
2071    2014  Air Force     244     167              350 -4.10      0        3
2422    2015  Air Force     225     190              344 -1.60      0        0
2773    2016  Air Force     214     171              296 -4.80      0        0
3124    2017  Air Force     271     220              341 -5.10      0        2


In [190]:
# append espn stats for winning teams to kaggle_df

for index, row in kaggle_df.iterrows():
    win_team = row["Wteamname"]
    season = row["Season"]
    espn_slice = espn_df[espn_df["Teamname"]==win_team]
    
    for i, r in espn_slice.iterrows():
        if r["Season"] == season:
            kaggle_df.set_value(index, "W_SOR_RK", r["SOR RK"])
            kaggle_df.set_value(index, "W_non_conf_sos", r["Non-Conf SOS RK"])
            kaggle_df.set_value(index, "W_SOS_RK", r["SOS RK"])
            kaggle_df.set_value(index, "W_BPI", r["BPI"])
            kaggle_df.set_value(index, "W_Q_wins", r["Q_wins"])
            kaggle_df.set_value(index, "W_Q_losses", r["Q_losses"])
    

In [191]:
# append esp stats for losing teams to kaggle_df

for index, row in kaggle_df.iterrows():
    lose_team = row["Lteamname"]
    season = row["Season"]
    espn_slice = espn_df[espn_df["Teamname"]==lose_team]
    
    for i, r in espn_slice.iterrows():
        if r["Season"] == season:
            kaggle_df.set_value(index, "L_SOR_RK", r["SOR RK"])
            kaggle_df.set_value(index, "L_non_conf_sos", r["Non-Conf SOS RK"])
            kaggle_df.set_value(index, "L_SOS_RK", r["SOS RK"])
            kaggle_df.set_value(index, "L_BPI", r["BPI"])
            kaggle_df.set_value(index, "L_Q_wins", r["Q_wins"])
            kaggle_df.set_value(index, "L_Q_losses", r["Q_losses"])

In [192]:
final_df = kaggle_df.dropna(how='any')
final_df.tail()

,Season,Daynum,Wteam,Wscore,Lteam,Lscore,Wor,Wdr,Wast,Wto,...,W_SOS_RK,W_BPI,W_Q_wins,W_Q_losses,L_SOR_RK,L_non_conf_sos,L_SOS_RK,L_BPI,L_Q_wins,L_Q_losses
53562,2017,132,1246,82,1116,65,12,28,10,15,...,206.00,-2.90,0,3,329.00,19.00,192.00,-9.60,0,4
53563,2017,132,1276,71,1458,56,3,22,17,9,...,137.00,0.10,0,2,22.00,27.00,40.00,15.20,10,6
53564,2017,132,1343,71,1463,59,9,23,15,7,...,164.00,8.20,0,1,107.00,154.00,152.00,1.80,0,1
53566,2017,132,1374,71,1153,56,7,27,14,8,...,83.00,15.10,3,2,16.00,103.00,81.00,15.60,5,3
53567,2017,132,1407,59,1402,53,14,19,5,5,...,234.00,2.10,0,0,150.00,350.00,292.00,-0.70,0,0


In [193]:
final_df.to_csv("../proc_data/FinalStats.csv", header=True)